In [1]:
import pandas as pd

In [2]:
cd C:\Users\jared\OneDrive\Documents\Personal\Independent_Projects\MLB smile

C:\Users\jared\OneDrive\Documents\Personal\Independent_Projects\MLB smile


In [3]:
mlb = pd.read_excel("mlbSmile_database.xlsx")
mlb2018 = mlb[mlb["Year"]==2018]
mlb2019 = mlb[mlb["Year"]==2019]
mlb2020 = mlb[mlb["Year"]==2020]
mlb_18_19 = mlb[(mlb["Year"].isin(['2018', '2019']))]
mlb_19_20 = mlb[(mlb["Year"].isin(['2019', '2020']))]
mlb_unique = mlb.drop_duplicates(subset=['mlb_id'], keep='first')

teamData = pd.read_excel("mlbSmile_database.xlsx", sheet_name = "Teams")

byState = pd.read_excel("mlbSmile_database.xlsx", sheet_name = "State")

In [4]:
#Group by Position
print((mlb.groupby(by=['Role'])['Cheese'].mean()*100).round(2))
print(f"\nBoth years combined, {round(mlb['Cheese'].mean()*100,1)}% of all MLB players smile in their headshots.")
print(f"Pitchers Smile {round(mlb[mlb['Position'] == '1P']['Cheese'].mean()*100,1)}% of the time")
print(f"Catchers Smile {round(mlb[mlb['Position'] == '2C']['Cheese'].mean()*100,1)}% of the time")
print(f"Infielders Smile {round(mlb[mlb['Position'] == '3INF']['Cheese'].mean()*100,1)}% of the time")
print(f"Outfielders Smile {round(mlb[mlb['Position'] == '4OF']['Cheese'].mean()*100,1)}% of the time")



Role
1B          80.37
2B          84.21
3B          79.70
C           77.86
CF          83.96
DH          81.82
LF          81.04
RF          79.89
Reliever    73.85
SS          88.82
Starter     76.74
Name: Cheese, dtype: float64

Both years combined, 78.0% of all MLB players smile in their headshots.
Pitchers Smile 74.9% of the time
Catchers Smile 78.0% of the time
Infielders Smile 82.4% of the time
Outfielders Smile 81.9% of the time


In [19]:
#Group by Country

byCountry = mlb_unique.groupby('Country') \
       .agg({'Cheese':'mean','Country':'size'}) \
       .rename(columns={'sent':'mean_cheese','Country':'count'}) \

byCountry = byCountry[byCountry["count"]>=10].sort_values(by= "Cheese", ascending = False)

print(byCountry)
print("")

#Group by Division

byDivision = mlb_unique.groupby('Division') \
       .agg({'Cheese':'mean','Division':'size'}) \
       .rename(columns={'sent':'mean_cheese','Division':'count'}) \

byDivision = byDivision[byDivision["count"]>=2].sort_values(by= "Cheese", ascending = False)

print(byDivision)
print("")

#Group by Conference

byConference = mlb_unique.groupby('Conference') \
       .agg({'Cheese':'mean','Conference':'size'}) \
       .rename(columns={'sent':'mean_cheese','Conference':'count'}) \


byConference = byConference[byConference["Conference"]>=10].sort_values(by= "Cheese", ascending = False)

print(byConference)
print("")

#Group by School

bySchool = mlb_unique.groupby('School') \
       .agg({'Cheese':'mean','School':'size'}) \
       .rename(columns={'sent':'mean_cheese','School':'count'}) \
       .reset_index()

bySchool = bySchool[bySchool["count"]>=10].sort_values(by= "Cheese", ascending = False)

print(bySchool)
print("")

#Group by Region

byRegion = mlb_unique.groupby('Region') \
       .agg({'Cheese':'mean','Region':'size'}) \
       .rename(columns={'sent':'mean_cheese','Region':'count'}) \

byRegion = byRegion[byRegion["count"]>=2].sort_values(by= "Cheese", ascending = False)

print(byRegion)
print("")

                      Cheese  count
Country                            
Colombia            0.909091     11
Canada              0.866667     15
Venezuela           0.851562    128
Mexico              0.842105     19
Puerto Rico         0.814815     27
Cuba                0.812500     32
Dominican Republic  0.769231    182
USA                 0.752449   1123

            Cheese  count
Division                 
D2        0.847826     46
NAIA      0.842105     19
HS        0.779703    404
D1        0.746933    652
D3        0.714286      7
Juco      0.663366    101

                 Cheese  Conference
Conference                         
SEC                  88         127
ACC                  62          79
Pac 12               56          75
Big West             33          38
Big 12               30          40
Conference USA       24          33
Big East             23          34
Missouri Valley      22          26
West Coast           16          22
Big 10               15          2

In [47]:
#run a correlation between smiling percent as a team and wins
#teams = mlb.pivot_table(values = "Cheese",index = "Team", columns = "Year", aggfunc = "mean")


from scipy.stats import pearsonr

corr, _ = pearsonr(teamData["Smile %"], teamData["Rk"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.185


In [7]:
#Create a USA Plot of Smile Percentage by State

import chart_studio
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot,plot
init_notebook_mode(connected=True) 

mlb_dropped = mlb.drop_duplicates(subset=['mlb_id'], keep='first')
mlb_state = mlb_dropped.groupby("State")
mean_mlb_state = mlb_state["Cheese"].mean().round(3)
mean_mlb_state = mean_mlb_state.reset_index()


data = dict(type='choropleth',
            colorscale = 'blues',
            reversescale = False,
            locations = byState['State'],
            z = byState['State_Perc'],
            locationmode = 'USA-states',
            text = byState["Yes"].astype('str') + " out of " + byState["Total"].astype('str') ,
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
            colorbar = {'title':"Smile Percentage by State"}
            ) 

layout = dict(title = 'MLB Smile Percentage by State, 2018-20',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

choromap = go.Figure(data = [data],layout = layout)
plot(choromap,validate=False)

username = 'jskolnicki'
api_key = 'gEFD9XgBXeAUS30aSQSs'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py
py.plot(choromap, filename="mlbSmile_states",auto_open=True)

'https://plotly.com/~jskolnicki/1/'

In [11]:
mlb2019[mlb2019['Position'].isin(["1P"])].pivot_table(values = "Salary",index = "Cheese", aggfunc = "count", columns = "Status_cond")

Status_cond,Arb,Pre-Arb,Vet
Cheese,,,
0,19,76,31
1,85,328,59


In [128]:
#Smile changes over time

#2018-2019

dataset = mlb


mlb.Cheese_convert = False

for row in mlb.index:
    if mlb.loc[row, 'Cheese'] == 1:
        mlb.loc[row, 'Cheese_convert'] = "Y"
    else:
        mlb.loc[row, 'Cheese_convert'] = "N"
        

names = []
ids = []
dates = []
smile_progressions = []
for athlete in dataset.mlb_id.unique():
    athlete_df = dataset.loc[mlb.mlb_id == athlete]
    athlete_df = athlete_df.sort_values(by = 'Year')
    names.append(athlete_df.Name.values[0])
    ids.append(athlete)
    dates.append(athlete_df.Year.values)
     
    string_cheese = ''

    for item in athlete_df.Cheese_convert.values:
    
        string_cheese = string_cheese + str(item)
        
    smile_progressions.append(string_cheese)
results = pd.DataFrame({'Player_Name': names,
                        'mlb_id' : ids,
                       'Dates': dates,
                       'Smile_Progression': smile_progressions})


results["Smile_Progression"].value_counts()

Y      473
YYY    418
YY     325
N      140
NYY     76
YYN     69
YN      64
NY      59
YNY     46
NN      40
NNN     40
NYN     37
NNY     32
YNN     24
Name: Smile_Progression, dtype: int64

In [152]:
results_team = pd.merge(results,mlb,on ="mlb_id",how = 'left')[["Player_Name","mlb_id","Dates","Smile_Progression","Team"]]

results_team = results_team.drop_duplicates(subset = "mlb_id",keep= 'first')

results_team = results_team[results_team.Smile_Progression.isin(["NNN","NNY","NYN","NYY","YYY","YYN","YNY","YNN"])]

print(results_team['Smile_Progression'].value_counts(normalize=False))

results_team[results_team["Smile_Progression"] == "NNY"]["Player_Name"].to_list()

YYY    418
NYY     76
YYN     69
YNY     46
NNN     40
NYN     37
NNY     32
YNN     24
Name: Smile_Progression, dtype: int64


['David Fletcher',
 'Paul Blackburn',
 'Jordan Lyles',
 'Joakim Soria',
 'Eric Thames',
 'John Gant',
 'Yadier Molina',
 'Victor Caratini',
 'Jake Diekman',
 'Kenley Jansen',
 'Hyun-Jin Ryu',
 'Josh Donaldson',
 'Ben Gamel',
 'Bryce Harper',
 'Tanner Scott',
 'Joey Lucchesi',
 'Kirby Yates',
 'Jake Arrieta',
 'Austin Davis',
 'Chris Archer',
 'Anthony Banda',
 'Diego Castillo',
 'Eduardo Rodriguez',
 'Rafael Devers',
 'Dustin Pedroia',
 'Jesse Winker',
 'Kevin McCarthy',
 'Bubba Starling',
 'Miguel Sano',
 'Tim Anderson',
 'Justus Sheffield',
 'Thairo Estrada']

In [163]:
#Evaluating Performance

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

mlb_19_20[mlb_19_20["Position"].isin(["2C","3IF","4OF","4DH"])].groupby(["Cheese"]).apply(wavg, "AB", "wOBA")

Cheese
0    197.918916
1    211.564063
dtype: float64

In [44]:
teamData[teamData["Year"] == 2020][['Year', 'Team', 'Smile %', 'W']].sort_values(by= "W", ascending = False)

,Year,Team,Smile %,W
69,2020,Dodgers,0.410256,43
81,2020,Rays,0.936170,40
77,2020,Padres,0.976744,37
62,2020,Athletics,0.833333,36
87,2020,Twins,0.435897,36
64,2020,Braves,0.476190,35
88,2020,White Sox,0.738095,35
71,2020,Indians,0.743590,35
67,2020,Cubs,0.636364,34
89,2020,Yankees,0.651163,33
